##**Welcome**
in the second hands-on session of this tutorial *Deep dive into the simulation of quantum circuits with QX-simulator*!

This session should last until the end of the time allocated for the tutorial. Again, please do not hesitate to ask questions when needed.

In this part, we will first try to think a little bit about sparse simulators and sparse quantum state vectors. For that, we won't use QX-simulator, but use Python to discover how to apply a quantum gate to a sparse state vector.

In the last part, we will take a critical look at the performance of sparse simulators like QX-simulator when confronted with a variety of quantum circuits. For that, we've prepared some experiences to compare the performance of QX-simulator and Qiskit's Aer state vector simulator.


# General questions

For the following questions, feel free to use the Internet to find answers, but of course it's better to try first to think by yourself, with your own knowledge. These questions are here to get everybody familiar with sparse state vector simulation - if a question is easy for you, immediately skip to the next one :).

1. What is a hashtable and what is it used for?

In [ ]:
# Type your answer in a comment here.

In [ ]:
#@title Solution

# A hashtable is a data structure used for efficient implementation of
# sets or maps. They need the key type to be hashable. They maintain a number of buckets,
# each bucket containing multiple elements. The index of the bucket in which a given key is stored
# is given by the hash function. That way, you can perform insertions, deletions and retrieval in amortized constant
# asymptotic time complexity. That means that the THEORETICAL time it takes to perform those operations
# does not grow with the number of elements that the hashtable contains.

2. Python dictionaries are implemented as hashtables. Let's try and represent some quantum state vectors with them. We will represent quantum kets as bitstrings using Python strings, for instance `"101001"`. Python also has built-in complex numbers, for instance you can write `myComplexNumber = 2+3.5j`.

We are representing **sparse** state vectors, so do not add the zero amplitudes.

Write below a Python dictionary representing the $\left| + \right>$ state, the Bell pair state and the GHZ state.

In [ ]:
import math

# Write your answers below.

plus = {}

bell_pair = {}

ghz = {}

In [ ]:
#@title Solution
import math

plus = {"0": math.sqrt(.5), "1": math.sqrt(.5)}
bell_pair = {"00": math.sqrt(.5), "11": math.sqrt(.5)}
ghz = {"00000": math.sqrt(.5), "11111": math.sqrt(.5)}
print(plus)
print(bell_pair)
print(ghz)



{'0': 0.7071067811865476, '1': 0.7071067811865476}
{'00': 0.7071067811865476, '11': 0.7071067811865476}
{'00000': 0.7071067811865476, '11111': 0.7071067811865476}


**[Bonus]** What do you think of representing kets with strings? How can you do better?

In [ ]:
#@title Solution

# Strings are intrinsically inefficient for that purpose: they are made to store characters.
# In Python strings are unicode, one character uses a minumum of 1 byte, that is 8 bits.
# Storing bits in a string is therefore very wasteful in terms of memory space.
# A better alternative is to implement a proper bitset data structure, or to use
# ad-hoc integers with bitmasks and bitwise operations to do this.
# For instance, 9 corresponds to the ket "1001".

3. Now that we have states, let's define matrices. You know that matrices representing quantum gates are square, unitary matrices whose size is a power of two. For writing matrices we will use the famous `numpy` library. Write `numpy` arrays for the matrices of quantum gates X, Y, H and CNOT.

In [ ]:
import numpy as np
import math

# Write your answers below.

X = np.array([])

Y = np.array([])

H = np.array([])

CNOT = np.array([])

In [ ]:
#@title Solution
import numpy as np
import math

X = np.array([[0, 1], [1, 0]])

Y = np.array([[0, -1j], [1j, 0]])

H = math.sqrt(.5) * np.array([[1, 1], [1, -1]])

CNOT = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])

print(X)
print(Y)
print(H)
print(CNOT)


[[0 1]
 [1 0]]
[[ 0.+0.j -0.-1.j]
 [ 0.+1.j  0.+0.j]]
[[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]
[[1 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]]


Quantum gate application is not a directly multiplication between the gate's matrix and the state vector. Indeed, the gate is often represented as a "small" matrix, but the state vector is in fact of size $2^N$ where $N$ is the number of qubits. What this means, is that you need an extra parameter, the operand or qubit index, to know how to apply the gate. This tells you the index of the gate in a tensor product of identities.

Concretely, applying single-qubit gate $G$ to qubit $i$ starting from the right on a $N$ qubits system means multiplying the state vector by the matrix
$$I ⊗ I \otimes \cdots \otimes G \otimes \cdots \otimes I$$
where there are $i$ identities at the right of $G$.

As for applying a quantum gate to a *sparse* state vector, it works the same as for normal state vectors, except that you won't see the zero or near-zero amplitudes.

4. Let's try to apply a gate to our sparse state vectors. For now we will only care about single-qubit gates. Write a Python function `apply(num_qubits, state_vector, matrix, operand)` which returns a new state vector, result of the application of `matrix` to `state_vector`.

Note: we don't have to assume that the state vector is a proper quantum state vector, that is, that the sum of the squared modulus of the amplitudes is equal to 1. We also don't have to assume that the matrices are unitary. Your function should work in any case.

In [2]:
import typing
import numpy as np

def apply(num_qubits: int, state_vector: dict[str, complex], matrix: np.ndarray, operand: int):
  """Given a `state_vector` describing a quantum computer containing `num_qubits`, apply gate given by
    `matrix` to qubit index `operand`. Returns the new state vector after that gate application."""
  assert(matrix.shape == (2, 2))                                                         # Check that the input matrix is square 2x2
  assert(all(len(ket) == num_qubits for ket in state_vector.keys()))                     # Check that the input state vector dictionary is well-formed
  assert(all(all(c == '0' or c == '1' for c in ket) for ket in state_vector.keys()))     # Check that the input state vector dictionary is well-formed
  assert(0 <= operand and operand < num_qubits)                                          # Check that the operand corresponds to a real qubit

  # Write your answer here

  return {"0": 1.}

In [4]:
#@title Solution

import typing

def apply(num_qubits: int, state_vector: dict[str, complex], matrix: np.ndarray, operand: int):
  assert(matrix.shape == (2, 2))                                                         # Check that the input matrix is square 2x2
  assert(all(len(ket) == num_qubits for ket in state_vector.keys()))                     # Check that the input state vector dictionary is well-formed
  assert(all(all(c == '0' or c == '1' for c in ket) for ket in state_vector.keys()))     # Check that the input state vector dictionary is well-formed
  assert(0 <= operand and operand < num_qubits)                                          # Check that the operand corresponds to a real qubit

  result = {}

  for ket, amplitude in state_vector.items():
    index = len(ket) - operand - 1
    op = ket[index]
    matrixCol = matrix[:, 0 if op == '0' else 1]

    firstKet = ket[:index] + "0" + ket[index + 1:]

    secondKet = ket[:index] + "1" + ket[index + 1:]

    firstAmplitude = result.get(firstKet, 0.)
    result[firstKet] = firstAmplitude + amplitude * matrixCol[0]

    secondAmplitude = result.get(secondKet, 0.)
    result[secondKet] = secondAmplitude + amplitude * matrixCol[1]

  return result

You can check your `apply` function by running the cell below.

In [5]:
import math
ATOL = 0.00000001
def eq(left: dict[str, complex], right: dict[str, complex]):
  """ Check equality between two sparse state vectors. """
  common_keys = [k for k in left.keys() if k in right]

  if not all(abs(v) == 0 for k, v in left.items() if k not in common_keys):
    return False

  if not all(abs(v) == 0 for k, v in right.items() if k not in common_keys):
    return False

  for k in common_keys:
    if abs(left[k] - right[k]) > ATOL:
      return False

  return True


def checkResult(num_qubits, state_vector, matrix, operand, expected_output):
  """ Check the result of `apply` on those parameters, raise an exception if incorrect. """
  result = apply(num_qubits, state_vector, matrix, operand)
  is_correct = eq(result, expected_output)
  if is_correct:
    return

  raise Exception(f"ERROR: your function returned {result} for\n\n\tnum_qubits\n{num_qubits}\n\n\tstate_vector\n{state_vector}\n\n\tmatrix\n{matrix}\n\n\toperand\n{operand}\n\nbut should have returned\n{expected_output}")


checkResult(1, {"0": 1.}, np.array([[1, 0], [0, 1]]), 0, {"0": 1.})
checkResult(1, {"0": 1j}, np.array([[1, 0], [0, 1]]), 0, {"0": 1j})
checkResult(1, {"0": 1j}, np.array([[1j, 0], [0, 1j]]), 0, {"0": -1.})
checkResult(1, {"0": math.sqrt(.5), "1": math.sqrt(.5)}, math.sqrt(.5) * np.array([[1, 1], [1, -1]]), 0, {"0": 1.})
checkResult(1, {"0": 1.}, math.sqrt(.5) * np.array([[1, 1], [1, -1]]), 0, {"0": math.sqrt(.5), "1": math.sqrt(.5)})
checkResult(2, {"00": 1.}, np.array([[1, 0], [0, 1]]), 0, {"00": 1.})
checkResult(2, {"00": 1.}, np.array([[0, 1], [1, 0]]), 0, {"01": 1.})
checkResult(2, {"00": 1.}, np.array([[0, 1], [1, 0]]), 1, {"10": 1.})
checkResult(2, {"10": 1.}, np.array([[0, 1], [1, 0]]), 0, {"11": 1.})
checkResult(2, {"00": 1.}, math.sqrt(.5) * np.array([[1, 1], [1, -1]]), 1, {"00": math.sqrt(.5), "10": math.sqrt(.5)})
checkResult(5, {"00000": math.sqrt(.5), "11111": math.sqrt(.5)}, np.array([[0, 1], [1, 0]]), 2, {"00100": math.sqrt(.5), "11011": math.sqrt(.5)})

What do you need to do to keep the state sparse when possible, when applying this algorithm?

In [ ]:
#@title Solution

# When inserting the elements in the result dict, make sure those amplitudes are non-zero,
# or that their absolute value / module is above a certain threshold (since we're computing with approximations
# due to floating-point arithmetic).

**[Advanced bonus]** How do you do this for 2+ qubit gates?

In [ ]:
#@title Solution

# You need to extract the relevant bits (based on operands) in the correct order
# and insert/add amplitudes for as many kets as their are lines in the matrix...

# QX vs Qiskit

For the second part we would like to demonstrate the internal capabilities of QX-simulator against Qiskit.

## Install pre-requisites

In [ ]:
!pip install qxelarator
import qxelarator

!pip install qiskit qiskit-aer
from qiskit import execute, QuantumCircuit, Aer, transpile
import time
import zipfile

with zipfile.ZipFile('/content/QuantumAlgorithms.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/extracted_folder')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 12.6 MB/s eta 0:00:00


## Simulate 64 qubits, woow

QX can handle this easily but Qiskit takes a lot of time and we have to quit the execution because it will crash. Check out the printed message!

After force quiting, we need to run the pre-requisites cell again.


In [ ]:
#QX
with open("/content/extracted_folder/QuantumAlgorithms/cqasm/wow.qasm", "r") as file:
    content = file.read()
    elapsed = time.time()
    result = qxelarator.execute_string(content)
print("--- %s seconds Elapsed time QX---" % (time.time() - elapsed))
# result

#Qiskit
qc = QuantumCircuit.from_qasm_file("/content/extracted_folder/QuantumAlgorithms/openqasm/wow.qasm") # quantum_volume_15
elapsed = time.time()
simulator = Aer.get_backend("statevector_simulator") # qasm_simulator, statevector_simulator
# Transpile the circuit for the simulator
compiled_circuit = transpile(qc, simulator)
# Run the simulation
result = simulator.run(compiled_circuit).result()
print("--- %s seconds Elapsed time Qiskit---" % (time.time() - elapsed))
# Get the state vector from the results
statevector = result.get_statevector()
# statevector
#print(counts)

--- 0.02855062484741211 seconds Elapsed time QX---


## Quantum Volume

The following comparison might not be fair as we get different probability distribution each time. In this example a instance of 5-qubit Quantum Volume cicruit is executed `range = 100` times and the measuremnt outcome is sampled. We can see distrubutions between Qiksit and QX are different each time we run the cell.

Regardless, what do you observe the elapsed time? How can you explain the result?

Quantum Volume circuits are random by nature and its type gate ratios are not in our control. Therefore, in the upcoming examples we have handpicked our quantum algorithm to be identical for cqasm and openqasm.

In [ ]:
range = 100 # interation times
rang = [0]*range

#Qiskit

qc = QuantumCircuit.from_qasm_file("/content/extracted_folder/QuantumAlgorithms/openqasm/quantum_volume_5.qasm") # quantum_volume_15
# You can choose other backend also.
elapsed = time.time()
counts_dict = {}
for i in rang:
    simulator = Aer.get_backend("statevector_simulator") # qasm_simulator, statevector_simulator
    # Transpile the circuit for the simulator
    compiled_circuit = transpile(qc, simulator)
    # Run the simulation
    result = simulator.run(compiled_circuit).result()
    counts = result.get_counts()
    if list(counts.keys())[0] in counts_dict:  # Check if the key exists in the dictionary
        counts_dict[list(counts.keys())[0]] += list(counts.values())[0]
    else:
        counts_dict[list(counts.keys())[0]] = list(counts.values())[0]
print("--- %s seconds Elapsed time Qiskit---" % (time.time() - elapsed))
# Get the state vector from the results
statevector = result.get_statevector()
print(counts_dict)
statevector
#print(counts)
print(statevector)

# plot_state_qsphere(statevector)

#QX
with open("/content/extracted_folder/QuantumAlgorithms/cqasm/quantum_volume_5.qasm", "r") as file: # quantum_volume_15
    content = file.read()
    # print(content)
    elapsed = time.time()
    result = qxelarator.execute_string(content,iterations=range)
    print("--- %s seconds Elapsed time QX---" % (time.time() - elapsed))
result


--- 16.74353528022766 seconds Elapsed time Qiskit---
{'10111': 2, '11110': 4, '11111': 13, '00000': 4, '00111': 1, '11101': 6, '01111': 16, '10001': 5, '01100': 2, '10000': 4, '00011': 5, '01011': 3, '00100': 2, '01101': 6, '11010': 2, '10101': 1, '11011': 4, '10100': 4, '00101': 4, '00010': 1, '10011': 3, '00110': 4, '01110': 3, '01010': 1}
Statevector([ 0.        +0.j       ,  0.        +0.j       ,
              0.        +0.j       , -0.54672612+0.8373115j,
              0.        +0.j       , -0.        +0.j       ,
              0.        +0.j       ,  0.        +0.j       ,
              0.        +0.j       ,  0.        +0.j       ,
             -0.        +0.j       , -0.        +0.j       ,
              0.        +0.j       ,  0.        +0.j       ,
              0.        +0.j       , -0.        +0.j       ,
              0.        +0.j       ,  0.        +0.j       ,
              0.        +0.j       , -0.        +0.j       ,
             -0.        +0.j       ,  0.      

Shots requested: 100
Shots done: 100
Results: {'00001': 4, '00010': 2, '00011': 8, '00100': 2, '00101': 4, '00110': 1, '00111': 1, '01000': 1, '01001': 1, '01010': 1, '01011': 6, '01100': 2, '01101': 6, '01110': 11, '01111': 25, '10000': 2, '10001': 6, '10010': 1, '10011': 5, '10100': 2, '10101': 1, '10110': 1, '10111': 2, '11000': 3, '11101': 2}
State: {'00011': (-0.46860680227018026-0.8834068512673626j)}

## Quantum Algorithm comparisons

Here we compare performances of QX versus Qiskit for different algorithms.
For Bernstein-Vazirani of 5 and 10 qubits (`bern_5, bern_10`), QX is faster. Also, both QFT algorithms (`qft_5, qft_15`) are faster in QX but for more than 15 qubits (`25` for example) is slower.


In [ ]:
range = 100 # interation times
rang = [0]*range

#Qiskit
qc = QuantumCircuit.from_qasm_file("/content/extracted_folder/QuantumAlgorithms/openqasm/bern_10.qasm") # qft_5, qft_15, qft_25, bern_5, bern_10
elapsed = time.time()
for i in rang:
    simulator = Aer.get_backend("statevector_simulator") # qasm_simulator, statevector_simulator
    # Transpile the circuit for the simulator
    compiled_circuit = transpile(qc, simulator)
    # Run the simulation
    result = simulator.run(compiled_circuit).result()
print("--- %s seconds Elapsed time Qiskit---" % (time.time() - elapsed))
# Get the state vector from the results
statevector = result.get_statevector()
#counts = result.get_counts()
#print(counts)
print(statevector)

# plot_state_qsphere(statevector)

#QX
with open("/content/extracted_folder/QuantumAlgorithms/cqasm/bern_10.qasm", "r") as file: # qft_5, qft_15, qft_25, bern_5, bern_1
    content = file.read()
    elapsed = time.time()
    result = qxelarator.execute_string(content,iterations = range)
print("--- %s seconds Elapsed time QX---" % (time.time() - elapsed))
result


--- 14.064067125320435 seconds Elapsed time Qiskit---
Statevector([ 4.25007252e-17-4.32978028e-17j,
              3.55724073e-17-6.74824734e-33j,
             -4.33680869e-18+7.20533824e-33j, ...,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2))
--- 1.6668477058410645 seconds Elapsed time QX---


Shots requested: 1000
Shots done: 1000
Results: {'00000000000': 1000}
State: {'01101101011': (0.7071067811865464+0j), '11101101011': (-0.7071067811865464+0j)}